# Chapter 2.  Matrices and Gaussian Elimination

In [5]:
# numerical and scientific computing libraries
import numpy as np
import scipy.linalg as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

--------------------------------------------------------------------------------------------------------------

### System of Linear Equations and $LU$ decomposition

Suppose that we are given a system of linear equations
\begin{align*}
\left\{ \begin{array}{ccl}
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n} x_n &=& b_1 \\
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n} x_n &=& b_2 \\
&\vdots& \\
a_{m1}x_1 + a_{m2}x_2 + \dots + a_{mn} x_n &=& b_m
\end{array} .\right.
\end{align*}

&nbsp;

Recalling how matrix multiplication is defined, we can rewrite this system into 
\begin{align*} 
A \mathbf{x} = \mathbf{b} 
\end{align*} 
where
\begin{align*}
A = \begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1n}  \\
a_{21} & a_{22} & \dots & a_{2n}  \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \dots & a_{mn}  
\end{bmatrix} , \quad \mathbf{x} = \begin{pmatrix} x_1 \\ x_2 \\ \vdots \\ x_n
\end{pmatrix} , \quad \mathbf{b} = \begin{pmatrix} b_1 \\ b_2 \\ \vdots \\ b_m
\end{pmatrix} .
\end{align*}

&nbsp;

To work with a concrete example, let us consider the case where $A$ is a square matrix that is invertible.

First, let us generate a random instance of this equation.

--------------------------------------------------------------------------------------------------------------

In [7]:
m = n = 10
rng = np.random.RandomState(0)

A = rng.randint(10, size=(m, n))
b = rng.randint(10, size=m)

print("A =")
print(A)
print("b =")
print(b)


A =
[[5 0 3 3 7 9 3 5 2 4]
 [7 6 8 8 1 6 7 7 8 1]
 [5 9 8 9 4 3 0 3 5 0]
 [2 3 8 1 3 3 3 7 0 1]
 [9 9 0 4 7 3 2 7 2 0]
 [0 4 5 5 6 8 4 1 4 9]
 [8 1 1 7 9 9 3 6 7 2]
 [0 3 5 9 4 4 6 4 4 3]
 [4 4 8 4 3 7 5 5 0 1]
 [5 9 3 0 5 0 1 2 4 2]]
b =
[0 3 2 0 7 5 9 0 2 7]


Let us begin with computing the $LU$-decomposition of $A$.

Recall that in general, during the computation of an $LU$-decomposition, one should perform row exchanges due to the positions of the pivots, which results in the need of introducing a permutation matrix $Q$ as an additional factor; $QA = LU$.   

SciPy provides the function `lu` as a member of the `linalg` submodule, but how it deals with the permutation matrix $Q$ slightly differs from our notations. When given a matrix $A$ as an input, the function `lu` returns a permutation matrix $P$, a lower triangular matrix $L$, and an upper triangular matrix $U$, but those three matrices satisfy $A = PLU$.

For a permutation matrix $Q$, it holds that $Q^{-1} = Q^\top$; see Appendix A in our book. Thus, if one wants an $LU$-decomposition in the form of $QA = LU$, one should take the transpose of $P$ as $Q$ in our text book after using the function `lu`, that is, $Q=P^\top$.

In [9]:
P, L, U = sp.lu(A)
Q = P.T

print("Q =")
print(Q)
print()
print("L =")
print(L)
print()
print("U =")
print(U)

Q =
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

L =
[[ 1.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.8889  1.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.5556 -0.5714  1.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.2222 -0.1429  0.95    1.      0.      0.      0.      0.      0.      0.    ]
 [ 0.     -0.5714  0.65   -0.1665  1.      0.      0.      0.      0.      0.    ]
 [ 0.7778  0.1429  0.9167  0.4698 -0.9975  1.      0.      0.      0.      0.    ]
 [ 0.     -0.4286  0.6333 -0.6407  0.6534 -0.3818  1.      0.      0.      0.    ]
 [ 0.5556 -0.5714  0.4167  0.5059  0.2876 -0.2433 -0.0055  1.      0.      0.  

In [10]:
# sanity check
np.allclose(Q@A, L@U)

True

Computing the $LDU$ decomposition of $A$ can be done by "extracting" the pivots from $U$.

In [11]:
# for simplicity, let us assume that A is invertible.
V = np.copy(U)
d = np.zeros(m)
for i in range(m):
    d[i] = V[i, i]
    U[i, :] = V[i, :] / d[i]

D = np.diag(d)

print("D =")
print(D)
print()
print("New U =")
print(U)

D =
[[ 9.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.     -7.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      8.5714  0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.     -7.7056  0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      6.5213  0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      7.0469  0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      8.887   0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.     -5.0091  0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     -5.6224  0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      1.2171]]

New U =
[[ 1.      1.      0.      0.4444  0.7778  0.3333  0.2222  0.7778  0.2222  0.    ]
 [-0.      1.     -0.1429 -0.4921 -0.3968 -0.9048 -0.1746  0.0317 -0.746 

In [12]:
# sanity check
np.allclose(Q @ A, L @ D @ U)

True

Now let us actually solve the equation $A \mathbf{x} = \mathbf{b}$. From $QA = LDU$, we have $LDU\mathbf{x} = QA \mathbf{x} = Q\mathbf{b}$. 
Let $\mathbf{y} = Q \mathbf{b}$.

In [13]:
y = Q @ b

We now need access to $L^{-1}$, or at least to $L^{-1} \mathbf{y}$. One possible approach is to perform Gaussian elimination on the augmented matrix $[L | \mathbf{y}]$, which will give us $[I | L^{-1} \mathbf{y}]$.

The code below shows a way to efficiently perform Gaussian elimination on $[L | \mathbf{y}]$, exploiting the structure of $L$.

**Verify by yourself that the code below is indeed a valid execution of Gaussian elimination.**

In [14]:
aug = np.hstack((L, y.reshape(-1, 1)))
# Because of the structure of L,
# the pivot is always on the main diagonal,
# and is equal to 1 every step.
for piv in range(m):
    for i in range(piv+1, m):
        aug[i, -1] = aug[i, -1] - aug[i, piv] * aug[piv, -1]
        aug[i, piv] = 0

L_inv_y = aug[:, -1]

In [15]:
# sanity check 1 : is Gaussian elimination correctly done?
print(aug)

[[ 1.      0.      0.      0.      0.      0.      0.      0.      0.      0.      7.    ]
 [ 0.      1.      0.      0.      0.      0.      0.      0.      0.      0.      2.7778]
 [ 0.      0.      1.      0.      0.      0.      0.      0.      0.      0.     -0.3016]
 [ 0.      0.      0.      1.      0.      0.      0.      0.      0.      0.     -0.8722]
 [ 0.      0.      0.      0.      1.      0.      0.      0.      0.      0.      6.6381]
 [ 0.      0.      0.      0.      0.      1.      0.      0.      0.      0.      4.4667]
 [ 0.      0.      0.      0.      0.      0.      1.      0.      0.      0.     -1.809 ]
 [ 0.      0.      0.      0.      0.      0.      0.      1.      0.      0.      4.4331]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      1.      0.     -2.0608]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      1.     -6.3442]]


In [16]:
# sanity check 2 : is L^(-1) y computed correctly?
np.allclose(L_inv_y, np.linalg.inv(L)@y)

True

We should now solve $DU \mathbf{x} = L^{-1} \mathbf{y}$. Because $D$ is a diagonal matrix, multiplying $D^{-1}$ with $L^{-1} \mathbf{y}$ is easy: notice that
\begin{align*}
[D^{-1}L^{-1} \mathbf{y}]_i = \frac{1}{d_{ii}} [L^{-1} \mathbf{y}]_i
\end{align*}
for all $i$, as you should verify.

In [17]:
D_inv_L_inv_y = np.zeros(m)
for i in range(m):
    D_inv_L_inv_y[i] = 1.0 / D[i, i] * L_inv_y[i]

For convenience, let $\mathbf{z} = D^{-1} L^{-1} \mathbf{y}$. We are then left with
\begin{align*}
U \mathbf{x} = \mathbf{z}.
\end{align*}

As $U$ is upper triangular, we can do back substitution. Mathematically, as the above translates to   
\begin{align*}
\sum_{j = i}^{n} u_{ij} x_j = z_i, \qquad i = 1, 2, \dots, n,
\end{align*}
by appropriately rearraging the terms we see that
\begin{align*}
x_i = \frac{1}{u_{ii}} \left( z_i - \sum_{j = i+1}^{n} u_{ij} x_j \right), \qquad i = n, n-1, \dots, 1.
\end{align*}

In [18]:
z = D_inv_L_inv_y
x = np.zeros(n)

for i in range(n-1, -1, -1):
    x[i] = (z[i] - sum(U[i, j] * x[j] for j in range(i+1, n))) / U[i, i]

print(x)

[-6.2381  4.187  -2.4684 -2.8606  0.4127  5.8132 -0.6281  1.563   3.4439 -5.2124]


In [19]:
# Final check
print("Ax =")
print(A @ x)
print()
print("b =")
print(b)

Ax =
[0. 3. 2. 0. 7. 5. 9. 0. 2. 7.]

b =
[0 3 2 0 7 5 9 0 2 7]
